# Agentic AI

In [1]:
from IPython.display import display, Math, Latex

import dspy

In [2]:
lm = dspy.LM('openai/gpt-4o-mini', api_key='API_KEY')

dspy.configure(lm=lm)

### COT le pide al LLM que argumente su resultado

In [3]:
math = dspy.ChainOfThought("question -> answer: float")
output = math(question="Two dice are tossed. What is the probability that the sum equals two?")
display(Latex(output.reasoning))

<IPython.core.display.Latex object>

### RAG define como contexto de búsqueda wiki17, con top-k resultados. En base a estos 3 resultados, el modelo responde.

In [4]:
def search_wikipedia(query: str) -> list[str]:
    results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=3)
    return [x['text'] for x in results]

rag = dspy.ChainOfThought('context, question -> response')

question = "What's the name of the castle that David Gregory inherited?"
rag(context=search_wikipedia(question), question=question)

Prediction(
    reasoning='The context mentions that David Gregory inherited Kinnairdy Castle in 1664. This information directly answers the question regarding the name of the castle he inherited.',
    response='The name of the castle that David Gregory inherited is Kinnairdy Castle.'
)

### ReAct define herramientas para ejecutar acciones. Tiene entonces acceso al ambiente (search) y puede tomar acciones (evaluate), lo cual lo define como un agente. 

In [5]:
def evaluate_math(expression: str):
    return dspy.PythonInterpreter({}).execute(expression)

react = dspy.ReAct("question -> answer: float", tools=[evaluate_math, search_wikipedia])

pred = react(question="What is 9362158 divided by the year of birth of David Gregory of Kinnairdy castle?")
print(pred.answer)

5761.328
